In [167]:
import pandas as pd
import numpy as np
import os, pickle, datetime
from sklearn.preprocessing import MinMaxScaler

In [168]:
# Load and combine the files
series = pd.read_csv('./WDISeries.csv')
series = series[['Series Code', 'Topic', 'Long definition', 
                 'Unit of measure', 'Periodicity', 'Base Period']]
countries = pd.read_csv('./WDICountry.csv')
countries.dropna(subset = ['Currency Unit'], inplace=True)
countries = countries[['Country Code', 'Short Name', 'Region', 'Income Group']]
df = pd.read_csv('./WDICSV.csv')
df = pd.merge(df, series, left_on=['Indicator Code'], right_on=['Series Code'], how='left')
df = pd.merge(df, countries, left_on=['Country Code'], right_on=['Country Code'], how='inner')

In [169]:
# Transform the data so that each metric is in its own column
metrics = [
    'Life expectancy at birth, total (years)',
    'Life expectancy at birth, female (years)',
    'Compulsory education, duration (years)',
    'Educational attainment, at least completed primary, population 25+ years, total (%) (cumulative)',
    'Educational attainment, at least completed lower secondary, population 25+, total (%) (cumulative)',
    'Educational attainment, at least completed upper secondary, population 25+, total (%) (cumulative)',
    'Educational attainment, at least completed primary, population 25+ years, female (%) (cumulative)',
    'Educational attainment, at least completed lower secondary, population 25+, female (%) (cumulative)',
    'Educational attainment, at least completed upper secondary, population 25+, female (%) (cumulative)',
    'GNI per capita, PPP (current international $)',
    'GDP, PPP (current international $)'
]

for metric in metrics:
    df[metric] = np.where(df['Indicator Name'] == metric, df['2023'], np.nan)

    # try to fill in missing values with previous years of data
    for year in range(2022,2014,-1):
        mask = (df['Indicator Name'] == metric) & (pd.isna(df[metric]))
        df.loc[mask, metric] = df.loc[mask, f'{year}']

cols_to_keep = ['Country Code'] + metrics

df_metrics = df[cols_to_keep].groupby(['Country Code']).agg('mean').reset_index()
df_metrics.shape

(217, 12)

In [170]:
df_metrics.dropna(inplace=True)
df_metrics.shape
# 46 countries dropped due to missing data

(171, 12)

In [171]:
# Sample 100 countries from the data
df_sample = df_metrics.sample(n=100, random_state=123)

In [172]:
# Add log-income metrics
df_sample['Log_GNI per capita, PPP'] = np.log(df_sample['GNI per capita, PPP (current international $)'])
df_sample['Log_GDP, PPP'] = np.log(df_sample['GDP, PPP (current international $)'])

In [173]:
metrics.append('Log_GNI per capita, PPP')
metrics.append('Log_GDP, PPP')

# Apply MinMaxScaler to the metrics
scaler = MinMaxScaler()
df_relative = df_sample.copy()
df_relative[metrics] = scaler.fit_transform(df_relative[metrics])

In [174]:
# Now we use the UN cutoffs for alternative HDI estimates
df_absolute = df_sample.copy()

# https://hdr.undp.org/sites/default/files/2023-24_HDR/hdr2023-24_technical_notes.pdf
# Life expectancy cutoff is 20 to 85
for var in ['Life expectancy at birth, total (years)','Life expectancy at birth, female (years)']:
    df_absolute[var] = (df_absolute[var] - 20) / (85 - 20)

# Education is 0 to 15 years
for var in ['Compulsory education, duration (years)']:
    df_absolute[var] = (df_absolute[var] - 0) / (15 - 0)

# Education attainment is 0 to 100%
for var in ['Educational attainment, at least completed primary, population 25+ years, total (%) (cumulative)',
            'Educational attainment, at least completed lower secondary, population 25+, total (%) (cumulative)',
            'Educational attainment, at least completed upper secondary, population 25+, total (%) (cumulative)',
            'Educational attainment, at least completed primary, population 25+ years, female (%) (cumulative)',
            'Educational attainment, at least completed lower secondary, population 25+, female (%) (cumulative)',
            'Educational attainment, at least completed upper secondary, population 25+, female (%) (cumulative)']:
    df_absolute[var] = (df_absolute[var] - 0) / (100 - 0)

# Log Income uses actual min and max values
for var in ['Log_GNI per capita, PPP']:
    df_absolute[var] = (df_absolute[var] - df_absolute[var].min()) / (df_absolute[var].max() - df_absolute[var].min())

In [175]:
# Average the education metrics
for df in [df_relative, df_absolute]:
    df['Avg_Education'] = df[[
        'Compulsory education, duration (years)',
        'Educational attainment, at least completed primary, population 25+ years, total (%) (cumulative)',
        'Educational attainment, at least completed lower secondary, population 25+, total (%) (cumulative)',
        'Educational attainment, at least completed upper secondary, population 25+, total (%) (cumulative)'
    ]].mean(axis=1)

    df['Avg_Education_Female'] = df[[
        'Compulsory education, duration (years)',
        'Educational attainment, at least completed primary, population 25+ years, female (%) (cumulative)',
        'Educational attainment, at least completed lower secondary, population 25+, female (%) (cumulative)',
        'Educational attainment, at least completed upper secondary, population 25+, female (%) (cumulative)'
    ]].mean(axis=1)

In [176]:
# Now take the geometric mean to get HDI estimates
for df in [df_relative, df_absolute]:
    df['HDI'] = df[['Life expectancy at birth, total (years)','Avg_Education','Log_GNI per capita, PPP']].prod(axis=1) ** (1/3)

    df['HDI_Female'] = df[['Life expectancy at birth, female (years)','Avg_Education_Female','Log_GNI per capita, PPP']].prod(axis=1) ** (1/3)

In [177]:
# Put the results into a df
df_relative.rename(columns={
    'HDI': 'HDI_Relative',
    'HDI_Female': 'HDI_Relative_Female'
}, inplace=True)
df_relative = df_relative[['Country Code', 'HDI_Relative', 'HDI_Relative_Female']]

df_absolute.rename(columns={
    'HDI': 'HDI_Absolute',
    'HDI_Female': 'HDI_Absolute_Female'
}, inplace=True)
df_absolute = df_absolute[['Country Code', 'HDI_Absolute', 'HDI_Absolute_Female']]

df_results = pd.merge(df_relative, df_absolute, on='Country Code', how='inner')
df_results = pd.merge(df_results, countries, on='Country Code', how='inner')

In [178]:
df_results

,Country Code,HDI_Relative,HDI_Relative_Female,HDI_Absolute,HDI_Absolute_Female,Short Name,Region,Income Group
0,UGA,0.325586,0.331812,0.413595,0.408862,Uganda,Sub-Saharan Africa,Low income
1,DJI,0.329451,0.328935,0.460324,0.439210,Djibouti,"Middle East, North Africa, Afghanistan & Pakistan",Lower middle income
2,MYS,0.687522,0.701903,0.765454,0.774425,Malaysia,East Asia & Pacific,Upper middle income
3,NLD,0.873192,0.873604,0.907835,0.912834,Netherlands,Europe & Central Asia,High income
4,SUR,0.494131,0.528021,0.599610,0.616475,Suriname,Latin America & Caribbean,Upper middle income
...,...,...,...,...,...,...,...,...
95,TCA,0.764909,0.775686,0.819747,0.828469,Turks and Caicos Islands,Latin America & Caribbean,High income
96,GHA,0.390268,0.398079,0.517442,0.508064,Ghana,Sub-Saharan Africa,Lower middle income
97,STP,0.350777,0.382194,0.460331,0.471169,São Tomé and Principe,Sub-Saharan Africa,Lower middle income
98,EST,0.785846,0.822058,0.844689,0.867878,Estonia,Europe & Central Asia,High income
